In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
import pandas as pd
import numpy as np
from skimage import io
from PIL import Image
import os

from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
from sklearn.model_selection import train_test_split

In [0]:
def read_image(path, dtype=np.float32, color=True):
    """
    Read an image from a given path, the image is in (C, H, W) format and the range of its value is between [0, 255]
    :param path: The path of an image file
    :param dtype: data type of an image, default is float32
    :param color: If 'True', the number of channels is three, in this case, RGB
        If 'False', this function returns a gray scale image
    :return:
    """
    f = Image.open(path)
    try:
        if color:
            img = f.convert('RGB')
        else:
            img = f.convert('P')
        img = np.asarray(img, dtype=dtype)
    finally:
        if hasattr(f, 'close'):
            f.close()

    if img.ndim == 2:
        # reshape (H, W) -> (1, H, W)
        return img[np.newaxis]
    else:
        # transpose (H, W, C) -> (C, H, W)
        return img.transpose((2, 0, 1))

In [0]:
class RSNADataset(Dataset):
    def __init__(self, root_dir, img_id, transform=False, train=True):
        """
        Args:
        :param root_dir (string): Directory with all the images
        :param img_id (list): lists of image id
        :param train: if equals true, then read training set, so the output is image, mask and imgId
                      if equals false, then read testing set, so the output is image and imgId
        :param transform (callable, optional): Optional transform to be applied on a sample
        """
        self.root_dir = root_dir
        self.img_id = img_id
        self.transform = transform
        # self.tsf = Transform(opt.min_size, opt.max_size, train)

    def __len__(self):
        return len(self.img_id)

    def __getitem__(self, idx):
        img_path = os.path.join(self.root_dir, self.img_id[idx], 'image.png')
        bbox_path = os.path.join(self.root_dir, self.img_id[idx], 'boxes.npy')
        image = read_image(img_path, np.float32, True)
        if os.path.exists(bbox_path):
            bbox = np.load(bbox_path)
            label = np.zeros(len(bbox)).astype(np.int32)
            sample = {'img_id': self.img_id[idx], 'image':image.copy(), 'bbox':bbox, 'label': label}
        else:
            sample = {'img_id': self.img_id[idx], 'image':image.copy()}

        # if self.transform:
        #     sample = self.tsf(sample)

        return sample

In [0]:
root_dir = "/content/gdrive/My Drive/Hackathon/data_final"

img_ids = os.listdir(root_dir)
img_ids.sort()
train_id, val_id = train_test_split(img_ids, test_size=0.1, random_state=55, shuffle=True)

train_dataset = RSNADataset(root_dir=root_dir, img_id=train_id, transform=False, train=True)
train_loader = DataLoader(train_dataset, batch_size=100,
                        shuffle=True)
# val_dataset = RSNADataset(root_dir=root_dir, img_id=val_id, transform=True, train=True)
# val_loader = DataLoader(val_dataset, batch_size=batch_size,
#                           shuffle=shuffle, num_workers=num_workers, pin_memory=pin_memory)

In [0]:
for i_batch, sample in enumerate(train_loader):
    show_batch_train(sample)

NotADirectoryError: ignored

In [0]:
import pandas as pd
df = pd.read_csv('/content/3. Object Identification/stage_2_train_labels.csv')

FileNotFoundError: ignored